Upload Dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data_bersih_step1.csv to data_bersih_step1 (1).csv


Model Baseline

Load Dataset + Cek Kolom Penting

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("data_bersih_step1.csv")

print("Shape (rows, cols):", df.shape)
print("Kolom:", list(df.columns))
display(df.head())

# auto-detect kolom
WIL_CANDS   = [c for c in df.columns if any(k in c.lower() for k in ["kabupaten","kota","wilayah","daerah","lokasi","nama_kabupaten_kota"])]
WAKTU_CANDS = [c for c in df.columns if any(k in c.lower() for k in ["periode","tanggal","date","update","waktu"])]
KOM_CANDS   = [c for c in df.columns if any(k in c.lower() for k in ["komoditas","kategori","commodity","produk","item"])]
HARGA_CANDS = [c for c in df.columns if any(k in c.lower() for k in ["harga","jumlah","price","nilai","rp"])]


WIL   = WIL_CANDS[0] if WIL_CANDS else None
WAKTU = WAKTU_CANDS[0] if WAKTU_CANDS else None
KOM   = KOM_CANDS[0] if KOM_CANDS else None
HARGA = HARGA_CANDS[0] if HARGA_CANDS else None

print("\nKolom terdeteksi:")
print("WIL  :", WIL)
print("WAKTU:", WAKTU)
print("KOM  :", KOM)
print("HARGA:", HARGA)

if not all([WIL, WAKTU, KOM, HARGA]):
    raise ValueError("Kolom kunci belum ketemu otomatis. Cek nama kolom lalu set manual variabel WIL/WAKTU/KOM/HARGA.")


Shape (rows, cols): (15275, 11)
Kolom: ['id', 'id_index', 'kode_provinsi', 'nama_provinsi', 'kode_kabupaten_kota', 'nama_kabupaten_kota', 'periode_update', 'kategori', 'jumlah', 'satuan', 'tahun']


,id,id_index,kode_provinsi,nama_provinsi,kode_kabupaten_kota,nama_kabupaten_kota,periode_update,kategori,jumlah,satuan,tahun
0,1,11,35,JAWA TIMUR,3501,KABUPATEN PACITAN,2020-01,BERAS MEDIUM,8500,RP/KG,2020
1,1,21,35,JAWA TIMUR,3501,KABUPATEN PACITAN,2020-01,BERAS PREMIUM,10500,RP/KG,2020
2,1,31,35,JAWA TIMUR,3501,KABUPATEN PACITAN,2020-01,GABAH KERING GILING,5500,RP/KG,2020
3,1,41,35,JAWA TIMUR,3501,KABUPATEN PACITAN,2020-01,GABAH KERING PANEN,0,RP/KG,2020
4,1,51,35,JAWA TIMUR,3501,KABUPATEN PACITAN,2020-01,JAGUNG PIPIL KERING,4200,RP/KG,2020



Kolom terdeteksi:
WIL  : kode_kabupaten_kota
WAKTU: periode_update
KOM  : kategori
HARGA: jumlah


Baseline Preprocessing

In [ ]:
import pandas as pd
import numpy as np

df2 = df.copy()

# waktu -> bulan & tahun
df2[WAKTU] = pd.to_datetime(df2[WAKTU], errors="coerce")
df2["bulan"] = df2[WAKTU].dt.month
df2["tahun"] = df2[WAKTU].dt.year

# harga numerik
df2[HARGA] = pd.to_numeric(df2[HARGA], errors="coerce")

# buang yang tidak lengkap
df2 = df2.dropna(subset=[WIL, KOM, HARGA, "bulan", "tahun"]).copy()

# ambang mahal baseline: P75 per (komoditas, bulan)
thr = (
    df2.groupby([KOM, "bulan"])[HARGA]
       .quantile(0.75)
       .reset_index()
       .rename(columns={HARGA: "p75"})
)

df2 = df2.merge(thr, on=[KOM, "bulan"], how="left")
df2["MAHAL"] = df2[HARGA] >= df2["p75"]

print("Total baris:", len(df2))
print("Baris MAHAL:", int(df2["MAHAL"].sum()))
display(df2[[WIL, "bulan", "tahun", KOM, HARGA, "p75", "MAHAL"]].head(10))


Total baris: 15275
Baris MAHAL: 5618


,kode_kabupaten_kota,bulan,tahun,kategori,jumlah,p75,MAHAL
0,3501,1,2020,BERAS MEDIUM,8500,11017.75,False
1,3501,1,2020,BERAS PREMIUM,10500,12000.00,False
2,3501,1,2020,GABAH KERING GILING,5500,6812.50,False
3,3501,1,2020,GABAH KERING PANEN,0,5708.25,False
4,3501,1,2020,JAGUNG PIPIL KERING,4200,4941.25,False
5,3501,1,2020,KEDELAI,0,0.00,True
6,3502,1,2020,BERAS MEDIUM,8750,11017.75,False
7,3502,1,2020,BERAS PREMIUM,10500,12000.00,False
8,3502,1,2020,GABAH KERING GILING,5362,6812.50,False
9,3502,1,2020,GABAH KERING PANEN,4478,5708.25,False


Bentuk Transaksi Baseline

In [ ]:
df_mahal = df2[df2["MAHAL"]].copy()

df_mahal["item"] = df_mahal[KOM].astype(str).str.upper().str.strip() + "_MAHAL"
df_mahal["transaction_id"] = (
    df_mahal[WIL].astype(str).str.upper().str.strip() + "_" +
    df_mahal["bulan"].astype(str) + "_" +
    df_mahal["tahun"].astype(str)
)

basket = df_mahal.groupby("transaction_id")["item"].apply(lambda x: sorted(set(x))).reset_index()

print("Jumlah transaksi:", basket.shape[0])
display(basket.head(10))


Jumlah transaksi: 2018


,transaction_id,item
0,3501_10_2021,[KEDELAI_MAHAL]
1,3501_10_2023,"[BERAS PREMIUM_MAHAL, JAGUNG PIPIL KERING_MAHA..."
2,3501_10_2024,"[BERAS MEDIUM_MAHAL, BERAS PREMIUM_MAHAL, GABA..."
3,3501_11_2020,[KEDELAI_MAHAL]
4,3501_11_2021,[KEDELAI_MAHAL]
5,3501_11_2022,[KEDELAI_MAHAL]
6,3501_11_2023,"[BERAS MEDIUM_MAHAL, BERAS PREMIUM_MAHAL, GABA..."
7,3501_11_2024,"[BERAS MEDIUM_MAHAL, BERAS PREMIUM_MAHAL, GABA..."
8,3501_12_2020,[KEDELAI_MAHAL]
9,3501_12_2021,[KEDELAI_MAHAL]


Jalankan FP-Growth Baseline + Rules

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

!pip -q install mlxtend

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
import pandas as pd

transactions = basket["item"].tolist()

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_te = pd.DataFrame(te_ary, columns=te.columns_)

MIN_SUPPORT = 0.05
MIN_CONF = 0.60

freq_itemsets = fpgrowth(df_te, min_support=MIN_SUPPORT, use_colnames=True)
freq_itemsets = freq_itemsets.sort_values("support", ascending=False)

print("Frequent itemsets:", len(freq_itemsets))
display(freq_itemsets.head(15))

rules = association_rules(freq_itemsets, metric="confidence", min_threshold=MIN_CONF)
rules = rules.sort_values(["lift", "confidence", "support"], ascending=False)

# tampil rapi
rules_show = rules.copy()
rules_show["antecedents"] = rules_show["antecedents"].apply(lambda x: ", ".join(list(x)))
rules_show["consequents"] = rules_show["consequents"].apply(lambda x: ", ".join(list(x)))

print("Rules:", len(rules_show))
display(rules_show[["antecedents","consequents","support","confidence","lift"]].head(20))


Frequent itemsets: 63


,support,itemsets
0,0.976214,(KEDELAI_MAHAL)
1,0.322597,(JAGUNG PIPIL KERING_MAHAL)
2,0.319623,(BERAS PREMIUM_MAHAL)
3,0.317146,(BERAS MEDIUM_MAHAL)
4,0.316650,(GABAH KERING GILING_MAHAL)
5,0.316650,(GABAH KERING PANEN_MAHAL)
6,0.307235,"(JAGUNG PIPIL KERING_MAHAL, KEDELAI_MAHAL)"
7,0.304262,"(BERAS PREMIUM_MAHAL, KEDELAI_MAHAL)"
32,0.301784,"(GABAH KERING PANEN_MAHAL, KEDELAI_MAHAL)"
17,0.301288,"(KEDELAI_MAHAL, GABAH KERING GILING_MAHAL)"


Rules: 331


,antecedents,consequents,support,confidence,lift
153,"GABAH KERING GILING_MAHAL, BERAS MEDIUM_MAHAL","GABAH KERING PANEN_MAHAL, BERAS PREMIUM_MAHAL,...",0.254212,0.872449,3.242361
150,"GABAH KERING PANEN_MAHAL, BERAS PREMIUM_MAHAL,...","GABAH KERING GILING_MAHAL, BERAS MEDIUM_MAHAL",0.254212,0.944751,3.242361
146,"GABAH KERING GILING_MAHAL, KEDELAI_MAHAL, BERA...","GABAH KERING PANEN_MAHAL, BERAS PREMIUM_MAHAL",0.254212,0.916071,3.237534
157,"GABAH KERING PANEN_MAHAL, BERAS PREMIUM_MAHAL","GABAH KERING GILING_MAHAL, KEDELAI_MAHAL, BERA...",0.254212,0.898424,3.237534
88,"GABAH KERING PANEN_MAHAL, BERAS PREMIUM_MAHAL","BERAS MEDIUM_MAHAL, GABAH KERING GILING_MAHAL",0.265610,0.938704,3.221607
93,"BERAS MEDIUM_MAHAL, GABAH KERING GILING_MAHAL","GABAH KERING PANEN_MAHAL, BERAS PREMIUM_MAHAL",0.265610,0.911565,3.221607
144,"GABAH KERING PANEN_MAHAL, KEDELAI_MAHAL, BERAS...","BERAS PREMIUM_MAHAL, GABAH KERING GILING_MAHAL",0.254212,0.932727,3.217510
159,"BERAS PREMIUM_MAHAL, GABAH KERING GILING_MAHAL","GABAH KERING PANEN_MAHAL, KEDELAI_MAHAL, BERAS...",0.254212,0.876923,3.217510
151,"BERAS PREMIUM_MAHAL, KEDELAI_MAHAL, GABAH KERI...","GABAH KERING PANEN_MAHAL, BERAS MEDIUM_MAHAL",0.254212,0.921005,3.215552
152,"GABAH KERING PANEN_MAHAL, BERAS MEDIUM_MAHAL","BERAS PREMIUM_MAHAL, KEDELAI_MAHAL, GABAH KERI...",0.254212,0.887543,3.215552


Model Final

Setup Data + Fungsi Pembentuk Transaksi

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("data_bersih_step1.csv").copy()

WIL   = "kode_kabupaten_kota"
WAKTU = "periode_update"
KOM   = "kategori"
HARGA = "jumlah"

df[WAKTU] = pd.to_datetime(df[WAKTU], errors="coerce")
df[HARGA] = pd.to_numeric(df[HARGA], errors="coerce")
df = df.dropna(subset=[WIL, WAKTU, KOM, HARGA]).copy()

df["bulan"] = df[WAKTU].dt.month
df["tahun"] = df[WAKTU].dt.year

def build_basket(df_in, q_mahal=0.75):
    d = df_in.copy()

    # threshold MAHAL: quantile per (komoditas, bulan)
    thr = (
        d.groupby([KOM, "bulan"])[HARGA]
         .quantile(q_mahal)
         .reset_index()
         .rename(columns={HARGA: "thr"})
    )
    d = d.merge(thr, on=[KOM, "bulan"], how="left")
    d["MAHAL"] = d[HARGA] >= d["thr"]

    dm = d[d["MAHAL"]].copy()
    dm["item"] = dm[KOM].astype(str).str.upper().str.strip() + "_MAHAL"
    dm["transaction_id"] = (
        dm[WIL].astype(str).str.strip() + "_" +
        dm["bulan"].astype(str) + "_" +
        dm["tahun"].astype(str)
    )

    basket = dm.groupby("transaction_id")["item"].apply(lambda x: sorted(set(x))).reset_index()
    return basket

print("Shape:", df.shape)
display(df.head())


Shape: (15275, 12)


,id,id_index,kode_provinsi,nama_provinsi,kode_kabupaten_kota,nama_kabupaten_kota,periode_update,kategori,jumlah,satuan,tahun,bulan
0,1,11,35,JAWA TIMUR,3501,KABUPATEN PACITAN,2020-01-01,BERAS MEDIUM,8500,RP/KG,2020,1
1,1,21,35,JAWA TIMUR,3501,KABUPATEN PACITAN,2020-01-01,BERAS PREMIUM,10500,RP/KG,2020,1
2,1,31,35,JAWA TIMUR,3501,KABUPATEN PACITAN,2020-01-01,GABAH KERING GILING,5500,RP/KG,2020,1
3,1,41,35,JAWA TIMUR,3501,KABUPATEN PACITAN,2020-01-01,GABAH KERING PANEN,0,RP/KG,2020,1
4,1,51,35,JAWA TIMUR,3501,KABUPATEN PACITAN,2020-01-01,JAGUNG PIPIL KERING,4200,RP/KG,2020,1


Grid Tuning

In [ ]:
!pip -q install mlxtend

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

grid_q = [0.70, 0.75, 0.80]      # tuning definisi MAHAL
grid_sup = [0.03, 0.05, 0.07]    # tuning min_support (naikkan dikit biar rules gak kebanyakan)
grid_conf = [0.70, 0.80, 0.90]   # tuning min_confidence

rows = []

for q in grid_q:
    basket = build_basket(df, q_mahal=q)
    transactions = basket["item"].tolist()

    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    df_te = pd.DataFrame(te_ary, columns=te.columns_)

    for sup in grid_sup:
        freq = fpgrowth(df_te, min_support=sup, use_colnames=True)
        n_itemsets = len(freq)

        for conf in grid_conf:
            if n_itemsets == 0:
                rows.append([q, sup, conf, len(basket), 0, 0, 0, np.nan, np.nan])
                continue

            rules = association_rules(freq, metric="confidence", min_threshold=conf)

            # filter awal biar tidak noise: lift > 1
            rules_f = rules[rules["lift"] > 1.0].copy()

            rows.append([
                q, sup, conf,
                len(basket),
                n_itemsets,
                len(rules),
                len(rules_f),
                rules_f["lift"].mean() if len(rules_f) else np.nan,
                rules_f["confidence"].mean() if len(rules_f) else np.nan
            ])

tuning_table = pd.DataFrame(rows, columns=[
    "q_mahal","min_support","min_confidence",
    "n_transaksi","n_itemsets","n_rules_raw","n_rules_lift>1",
    "avg_lift","avg_conf"
])

# tampilkan kombinasi terbaik (prioritas: rules tidak kebanyakan + lift tinggi)
display(tuning_table.sort_values(["n_rules_lift>1","avg_lift"], ascending=False).head(15))


,q_mahal,min_support,min_confidence,n_transaksi,n_itemsets,n_rules_raw,n_rules_lift>1,avg_lift,avg_conf
9,0.75,0.03,0.7,2018,63,331,300,2.948251,0.881551
12,0.75,0.05,0.7,2018,63,331,300,2.948251,0.881551
15,0.75,0.07,0.7,2018,63,331,300,2.948251,0.881551
0,0.70,0.03,0.7,2077,63,331,300,2.558944,0.898851
3,0.70,0.05,0.7,2077,63,331,300,2.558944,0.898851
6,0.70,0.07,0.7,2077,63,331,300,2.558944,0.898851
1,0.70,0.03,0.8,2077,63,328,297,2.559380,0.899892
4,0.70,0.05,0.8,2077,63,328,297,2.559380,0.899892
7,0.70,0.07,0.8,2077,63,328,297,2.559380,0.899892
10,0.75,0.03,0.8,2018,63,305,274,2.961221,0.891660


Pilih Kombinasi Final

In [ ]:
cand = tuning_table[
    (tuning_table["n_rules_lift>1"] >= 30) &
    (tuning_table["n_rules_lift>1"] <= 200) &
    (tuning_table["avg_lift"].notna())
].sort_values(["avg_lift","avg_conf"], ascending=False)

display(cand.head(10))

# kalau tabel kosong, longgarkan batasnya:
if cand.empty:
    cand2 = tuning_table[tuning_table["n_rules_lift>1"] > 0].sort_values(["avg_lift"], ascending=False)
    print("Kandidat (batas dilonggarkan):")
    display(cand2.head(10))


,q_mahal,min_support,min_confidence,n_transaksi,n_itemsets,n_rules_raw,n_rules_lift>1,avg_lift,avg_conf
20,0.80,0.03,0.9,1703,63,53,53,3.108296,0.934854
23,0.80,0.05,0.9,1703,63,53,53,3.108296,0.934854
26,0.80,0.07,0.9,1703,55,40,40,3.083560,0.931647
19,0.80,0.03,0.8,1703,63,147,147,3.020118,0.881245
22,0.80,0.05,0.8,1703,63,147,147,3.020118,0.881245
18,0.80,0.03,0.7,1703,63,215,198,3.014184,0.850002
21,0.80,0.05,0.7,1703,63,215,198,3.014184,0.850002
24,0.80,0.07,0.7,1703,55,172,155,3.000499,0.848216
25,0.80,0.07,0.8,1703,55,116,116,2.996515,0.878751
11,0.75,0.03,0.9,2018,63,160,129,2.995703,0.932280


Jalankan Model Final + Filter Rules

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
import pandas as pd

# ====== SET PARAMETER FINAL (isi dari hasil cand) ======
Q_FINAL = 0.75
SUP_FINAL = 0.05
CONF_FINAL = 0.80

# filter tambahan untuk hasil final (biar aturan lebih bermakna)
LIFT_MIN = 1.10
MAX_ANT = 2   # max item di antecedent
MAX_CON = 1   # max item di consequent

basket_final = build_basket(df, q_mahal=Q_FINAL)
transactions_final = basket_final["item"].tolist()

te = TransactionEncoder()
te_ary = te.fit(transactions_final).transform(transactions_final)
df_te = pd.DataFrame(te_ary, columns=te.columns_)

freq_final = fpgrowth(df_te, min_support=SUP_FINAL, use_colnames=True)
rules_final = association_rules(freq_final, metric="confidence", min_threshold=CONF_FINAL)

# filter rules final
rules_final = rules_final[rules_final["lift"] >= LIFT_MIN].copy()
rules_final = rules_final[
    (rules_final["antecedents"].apply(lambda x: len(x)) <= MAX_ANT) &
    (rules_final["consequents"].apply(lambda x: len(x)) <= MAX_CON)
].sort_values(["lift","confidence","support"], ascending=False)

# buat tampilan rapi
rules_show = rules_final.copy()
rules_show["antecedents"] = rules_show["antecedents"].apply(lambda x: ", ".join(list(x)))
rules_show["consequents"] = rules_show["consequents"].apply(lambda x: ", ".join(list(x)))

print("=== MODEL FINAL FP-GROWTH ===")
print("Transaksi:", len(basket_final))
print("Frequent itemsets:", len(freq_final))
print("Rules final (filtered):", len(rules_show))

display(rules_show[["antecedents","consequents","support","confidence","lift"]].head(20))


=== MODEL FINAL FP-GROWTH ===
Transaksi: 2018
Frequent itemsets: 63
Rules final (filtered): 48


,antecedents,consequents,support,confidence,lift
122,"GABAH KERING PANEN_MAHAL, BERAS PREMIUM_MAHAL",BERAS MEDIUM_MAHAL,0.274529,0.970228,3.059249
29,"BERAS PREMIUM_MAHAL, GABAH KERING GILING_MAHAL",BERAS MEDIUM_MAHAL,0.280476,0.967521,3.050716
128,"GABAH KERING PANEN_MAHAL, BERAS PREMIUM_MAHAL",GABAH KERING GILING_MAHAL,0.273043,0.964974,3.047444
99,"GABAH KERING PANEN_MAHAL, BERAS MEDIUM_MAHAL",GABAH KERING GILING_MAHAL,0.275520,0.961938,3.037857
30,"BERAS PREMIUM_MAHAL, BERAS MEDIUM_MAHAL",GABAH KERING GILING_MAHAL,0.280476,0.956081,3.019361
31,"BERAS MEDIUM_MAHAL, GABAH KERING GILING_MAHAL",BERAS PREMIUM_MAHAL,0.280476,0.962585,3.011623
123,"GABAH KERING PANEN_MAHAL, BERAS MEDIUM_MAHAL",BERAS PREMIUM_MAHAL,0.274529,0.958478,2.998771
100,"BERAS MEDIUM_MAHAL, GABAH KERING GILING_MAHAL",GABAH KERING PANEN_MAHAL,0.275520,0.945578,2.986192
130,"BERAS PREMIUM_MAHAL, GABAH KERING GILING_MAHAL",GABAH KERING PANEN_MAHAL,0.273043,0.941880,2.974514
98,"GABAH KERING PANEN_MAHAL, GABAH KERING GILING_...",BERAS MEDIUM_MAHAL,0.275520,0.942373,2.971419
